In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn.metrics as skm 
import matplotlib.pyplot as plt

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# All

In [4]:
df_deepar = pd.read_csv("/content/DeepAR_base_pred_single.csv",)
df_nbeats = pd.read_csv("/content/NBeats_base_pred_single.csv")
df_tft = pd.read_csv("/content/TFT_base_pred_single.csv")

df_deepar.head()

,Unnamed: 0,id,pred_severity,DWT_dist,actual_severity
0,0,PN-204-pic-1-winterday.wav,0,0.007216,0
1,1,PN-068-pic-1-sportsman.wav,2,0.012236,3
2,2,PD-070-pic-1-winterday.wav,2,0.005900,0
3,3,PN-303-pic-1-sportsman.wav,0,0.009558,0
4,4,PD-115-pic-1-sportsman.wav,0,0.039050,0


In [5]:
df_deepar.drop(['Unnamed: 0'], axis=1, inplace=True)
df_nbeats.drop(['Unnamed: 0'], axis=1, inplace=True)
df_tft.drop(['Unnamed: 0'], axis=1, inplace=True)

In [9]:
report_deepar = skm.classification_report(df_deepar['actual_severity'], df_deepar['pred_severity'], output_dict=True)
report_nbeats = skm.classification_report(df_nbeats['actual_severity'], df_nbeats['pred_severity'], output_dict=True)
report_tft = skm.classification_report(df_tft['actual_severity'], df_tft['pred_severity'], output_dict=True)

acc_deepar = round(sum(df_deepar['pred_severity'] == df_deepar['actual_severity']) / len(df_deepar['actual_severity']), 3)
acc_nbeats = round(sum(df_nbeats['pred_severity'] == df_nbeats['actual_severity']) / len(df_nbeats['actual_severity']), 3)
acc_tft = round(sum(df_tft['pred_severity'] == df_tft['actual_severity']) / len(df_tft['actual_severity']), 3)

In [11]:
pd.DataFrame([[report_deepar['weighted avg']['f1-score'], report_nbeats['weighted avg']['f1-score'], report_tft['weighted avg']['f1-score']], 
              [report_deepar['weighted avg']['precision'], report_nbeats['weighted avg']['precision'], report_tft['weighted avg']['precision']],
              [report_deepar['weighted avg']['recall'], report_nbeats['weighted avg']['recall'], report_tft['weighted avg']['recall']],
              [acc_deepar, acc_nbeats, acc_tft]], index=['f1-score', 'precision', 'recall', 'accuracy'], columns=['DeepAR', 'N-Beats', 'TFT']).T

,f1-score,precision,recall,accuracy
DeepAR,0.349598,0.352183,0.354167,0.354
N-Beats,0.443642,0.457837,0.437500,0.438
TFT,0.356884,0.360417,0.354167,0.354


In [ ]:
report_deepar = skm.classification_report(df_deepar['actual_severity'], df_deepar['pred_severity'], output_dict=True)
report_nbeats = skm.classification_report(df_nbeats['actual_severity'], df_nbeats['pred_severity'], output_dict=True)
report_tft = skm.classification_report(df_tft['actual_severity'], df_tft['pred_severity'], output_dict=True)

acc_deepar = round(sum(df_deepar['pred_severity'] == df_deepar['actual_severity']) / len(df_deepar['actual_severity']), 3)
acc_nbeats = round(sum(df_nbeats['pred_severity'] == df_nbeats['actual_severity']) / len(df_nbeats['actual_severity']), 3)
acc_tft = round(sum(df_tft['pred_severity'] == df_tft['actual_severity']) / len(df_tft['actual_severity']), 3)

In [ ]:
pd.DataFrame([[report_deepar['0']['f1-score'], report_nbeats['0']['f1-score'], report_tft['0']['f1-score']], 
              [report_deepar['0']['precision'], report_nbeats['0']['precision'], report_tft['0']['precision']],
              [report_deepar['0']['recall'], report_nbeats['0']['recall'], report_tft['0']['recall']],
              [acc_deepar, acc_nbeats, acc_tft]], index=['f1-score', 'precision', 'recall', 'accuracy'], columns=['DeepAR', 'N-Beats', 'TFT']).T

,f1-score,precision,recall,accuracy
DeepAR,0.561404,0.571429,0.551724,0.479
N-Beats,0.631579,0.642857,0.620690,0.562
TFT,0.551724,0.551724,0.551724,0.458


## Models

In [81]:
deepar_cv = "/content/drive/MyDrive/Grid_search_thesis/DeepAR_models"
tft_cv = '/content/drive/MyDrive/Grid_search_thesis/TFT_models/CV_results'
nbeats_cv = "/content/drive/MyDrive/Grid_search_thesis/NBeats_models/CV_results"

In [89]:
types = ['narrative', 'story', 'instruction']

# all_models = {'nbeats_narrative' : [], 'nbeats_story' : [], 'nbeats_instruction' : [],
#               'tft_narrative' : [], 'tft_story' : [], 'tft_instruction' : [],
#               'deepar' : []}

for stimuli in types:
  f1 = []
  rec = []
  pre = []
  acc = []
  f1_avg = 0
  recall_avg = 0
  precision_avg = 0
  accuracy_avg = 0
  for i in range(10):
    # tft = pd.read_csv(os.path.join(tft_cv, f'TFT_{stimuli}_fold_{i}_pred.csv'))
    # report_tft = skm.classification_report(tft['actual_severity'], tft['pred_severity'], output_dict=True)

    nbeats = pd.read_csv(os.path.join(nbeats_cv, f'NBeats_{stimuli}_fold_{i}_pred.csv'))

    # For binary classification
    nbeats.loc[nbeats['pred_severity'] == 2, 'pred_severity'] = 1 
    nbeats.loc[nbeats['pred_severity'] == 3, 'pred_severity'] = 1

    nbeats.loc[nbeats['actual_severity'] == 2, 'actual_severity'] = 1
    nbeats.loc[nbeats['actual_severity'] == 3, 'actual_severity'] = 1

    report_nbeats = skm.classification_report(nbeats['actual_severity'], nbeats['pred_severity'], output_dict=True)
    # print(report_nbeats['weighted avg']['f1-score'])
    f1_avg += report_nbeats['weighted avg']['f1-score']
    recall_avg += report_nbeats['weighted avg']['recall']
    precision_avg += report_nbeats['weighted avg']['precision']
    accuracy_avg += report_nbeats['accuracy']
    f1.append(report_nbeats['weighted avg']['f1-score'])
    rec.append(report_nbeats['weighted avg']['recall'])
    pre.append(report_nbeats['weighted avg']['precision'])
    acc.append(report_nbeats['accuracy'])
  
  print(sum((np.array(f1) - (f1_avg / 10))**2) / 9)
  print(sum((np.array(rec) - (recall_avg / 10))**2) / 9)
  print(sum((np.array(pre) - (precision_avg / 10))**2) / 9)
  print(sum((np.array(acc) - (accuracy_avg / 10))**2) / 9)

  print(f"Nbeats {stimuli}:\n\t F1: {f1_avg / 10}\n\t Recall: {recall_avg / 10}\n\t Precision: {precision_avg / 10}\n\t Accuracy: {accuracy_avg / 10}")
    # all_models[f'nbeats_{stimuli}'] = report_nbeats['weighted avg']['f1-score'], report_tft['weighted avg']['f1-score']

0.0017878495434634246
0.0016203703703703703
0.0016053626599529807
0.0016203703703703703
Nbeats narrative:
	 F1: 0.6143363656530018
	 Recall: 0.6208333333333332
	 Precision: 0.6187576363651306
	 Accuracy: 0.6208333333333332
0.0018817014030730374
0.0018759645061728386
0.0019131048546583503
0.0018759645061728386
Nbeats story:
	 F1: 0.48147195711578006
	 Recall: 0.4770833333333334
	 Precision: 0.49230797872283383
	 Accuracy: 0.4770833333333334
0.0013366773872518986
0.0013695987654320993
0.0020556630343321245
0.0013695987654320993
Nbeats instruction:
	 F1: 0.5162738038094526
	 Recall: 0.5125
	 Precision: 0.5415433163885056
	 Accuracy: 0.5125


In [90]:
types = ['narrative', 'story', 'instruction']

# all_models = {'nbeats_narrative' : [], 'nbeats_story' : [], 'nbeats_instruction' : [],
#               'tft_narrative' : [], 'tft_story' : [], 'tft_instruction' : [],
#               'deepar' : []}

for stimuli in types:
  f1 = []
  rec = []
  pre = []
  acc = []
  f1_avg = 0
  recall_avg = 0
  precision_avg = 0
  accuracy_avg = 0
  for i in range(10):
    # tft = pd.read_csv(os.path.join(tft_cv, f'TFT_{stimuli}_fold_{i}_pred.csv'))
    # report_tft = skm.classification_report(tft['actual_severity'], tft['pred_severity'], output_dict=True)

    tft = pd.read_csv(os.path.join(tft_cv, f'TFT_{stimuli}_fold_{i}_pred.csv'))
    
    # For binary classification
    tft.loc[tft['pred_severity'] == 2, 'pred_severity'] = 1 
    tft.loc[tft['pred_severity'] == 3, 'pred_severity'] = 1

    tft.loc[tft['actual_severity'] == 2, 'actual_severity'] = 1
    tft.loc[tft['actual_severity'] == 3, 'actual_severity'] = 1

    report_tft = skm.classification_report(tft['actual_severity'], tft['pred_severity'], output_dict=True)
    # print(report_nbeats['weighted avg']['f1-score'])
    f1_avg += report_tft['weighted avg']['f1-score']
    recall_avg += report_tft['weighted avg']['recall']
    precision_avg += report_tft['weighted avg']['precision']
    accuracy_avg += report_tft['accuracy']
    f1.append(report_tft['weighted avg']['f1-score'])
    rec.append(report_tft['weighted avg']['recall'])
    pre.append(report_tft['weighted avg']['precision'])
    acc.append(report_tft['accuracy'])
  
  print(sum((np.array(f1) - (f1_avg / 10))**2) / 9)
  print(sum((np.array(rec) - (recall_avg / 10))**2) / 9)
  print(sum((np.array(pre) - (precision_avg / 10))**2) / 9)
  print(sum((np.array(acc) - (accuracy_avg / 10))**2) / 9)

  print(f"TFT {stimuli}:\n\t F1: {f1_avg / 10}\n\t Recall: {recall_avg / 10}\n\t Precision: {precision_avg / 10}\n\t Accuracy: {accuracy_avg / 10}")
    # all_models[f'nbeats_{stimuli}'] = report_nbeats['weighted avg']['f1-score'], report_tft['weighted avg']['f1-score']

0.001526606362275417
0.0015866126543209863
0.0017395399821316976
0.0015866126543209863
TFT narrative:
	 F1: 0.6559837504167639
	 Recall: 0.6645833333333332
	 Precision: 0.659607558071248
	 Accuracy: 0.6645833333333332
0.0017459600324489228
0.001176697530864198
0.001984393474029971
0.001176697530864198
TFT story:
	 F1: 0.5366098880759391
	 Recall: 0.5541666666666666
	 Precision: 0.531702105548094
	 Accuracy: 0.5541666666666666
0.001979022265249885
0.0020061728395061726
0.0021669739358047093
0.0020061728395061726
TFT instruction:
	 F1: 0.5628376907209622
	 Recall: 0.5583333333333333
	 Precision: 0.5755019139978379
	 Accuracy: 0.5583333333333333


In [39]:
deepar_cv = '/content/drive/MyDrive/Grid_search_thesis/DeepAR_models/cv'

f1 = []
rec = []
pre = []
acc = []
f1_avg = 0
recall_avg = 0
precision_avg = 0
accuracy_avg = 0

n = 10

for i in range(n):
  predictions = pd.read_csv(os.path.join(deepar_cv, f'deepar_fold_{i}_pred.csv'))

  report = skm.classification_report(predictions['actual_severity'], predictions['pred_severity'], output_dict=True)

  f1_avg += report['weighted avg']['f1-score']
  recall_avg += report['weighted avg']['recall']
  precision_avg += report['weighted avg']['precision']
  accuracy_avg += report['accuracy']
  f1.append(report['weighted avg']['f1-score'])
  rec.append(report['weighted avg']['recall'])
  pre.append(report['weighted avg']['precision'])
  acc.append(report['accuracy'])

print(sum((np.array(f1) - (f1_avg / n))**2) / (n-1))
print(sum((np.array(rec) - (recall_avg / n))**2) / (n-1))
print(sum((np.array(pre) - (precision_avg / n))**2) / (n-1))
print(sum((np.array(acc) - (accuracy_avg / n))**2) / (n-1))

print(f"Deepar:\n\t F1: {f1_avg / n}\n\t Recall: {recall_avg / n}\n\t Precision: {precision_avg / n}\n\t Accuracy: {accuracy_avg / n}")

0.0013034726151785877
0.0015432098765432094
0.0018559858485309537
0.0015432098765432094
Deepar:
	 F1: 0.5847752628975059
	 Recall: 0.625
	 Precision: 0.5650236377993731
	 Accuracy: 0.625
